In [5]:
import pyttsx3
import random
import json
import torch

from model import chatModel , chatModel_v2 ,chatModel_v3, chatModel_v3_extended
# from nltk_utils import bag_of_words, tokenize_fn
from func import preprocess_words_fn ,train_fn ,accuracy_fn, bag_of_words 

from func import convert_speach_to_text
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define stop words using nltk's stopword list
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)


# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Get available voices

voices = engine.getProperty('voices')       #getting details of current voice

# Set the voice to female
#engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female
                        
# Adjust other properties if needed
engine.setProperty('rate', 120)  # Speed of speech
engine.setProperty('volume', 1)  # Volume level (0.0 to 1.0)

with open('intents2.json', 'r') as f:
    intents = json.load(f)

datasetfile = "train_dataset_v2.pt"
data = torch.load(datasetfile)
# engine.say(f"how are you my name is {bot_name} . write you name ")
engine.say(f"how are you my name is {bot_name} . what is you name ")
engine.runAndWait()
# x = input("You: ")
x = convert_speach_to_text()
engine.say(f"how are you {x}  how can I help you")
engine.runAndWait()

tags=data.classes

model_1 = torch.load("modelv3e.pt")
model_1.eval()

bot_name = "Sara"
print("Let's chat! (type 'quit' to exit)")
topics = data.classes
print(f"topics:{data.classes}")
text = convert_speach_to_text()
last_prediction_tag = None
last_response_index = None
command = True



while command:

    text = convert_speach_to_text()
    # sent = input("You: ").lower()  
    sent = text
    if sent in ["quit", "q", "exit", "x"]:
        command = False
        engine.say(f"good bye {x}")
        engine.runAndWait()
   
    elif sent == "more" and last_predicted_tag is not None:
    # If "more" is typed, try to give another response
        for intent in intents['intents']:
            if last_predicted_tag == intent["tag"]:
                responses = intent['responses']
                last_response_index = (last_response_index + 1) % len(responses)
                print(f"{bot_name}: {responses[last_response_index]} \n")
                engine.say(responses[last_response_index])
                engine.runAndWait()
    else:
     # Normal flow: Predict the tag and respond
        bag = bag_of_words(sentence=sent ,all_words=data.all_words,
                       preprocess_words_fn= preprocess_words_fn,punctuation=punctuation,
                       stop_words=stop_words)
        bag_tensor= torch.tensor(bag, dtype = torch.float32).unsqueeze(0)
        prediction = model_1.predict(bag_tensor)  
        
    
        predicted_index = prediction.item()  # Extract the scalar value from the tensor
        predicted_tag = tags[predicted_index]  # Map the index to the tag name
        
        

        if predicted_tag in topics:
            for intent in intents['intents']:
                    if predicted_tag == intent["tag"]:
                        response = random.choice(intent['responses'])
                        print(f"{bot_name}: {response}\n")
                       
                        
                        engine.say(response)
                        engine.runAndWait()
                        # print(f"{bot_name}: {random.choice(intent['responses'])}")
                        # Save state for "more"
                        last_predicted_tag = predicted_tag
                        last_response_index = response.index(response)
        else:
            print(f" {bot_name}: I do not understand... \n")
            last_predicted_tag = None
            last_response_index = None
    
    

Adjusting for ambient noise... Please wait.
Please speak now...
You said:  Muhammad
Let's chat! (type 'quit' to exit)
topics:['greeting', 'goodbye', 'practical_programs', 'study_AI', 'study_tips', 'best_study_time', 'student_support']
Adjusting for ambient noise... Please wait.
Please speak now...
You said:  how to study AI
Adjusting for ambient noise... Please wait.
Please speak now...
You said:  how to study AI
Sara: Studying AI involves learning key subjects like Machine Learning, Data Science, and Artificial Intelligence theory. We offer a variety of programs that combine theory with practical projects, ensuring you can apply what you learn. You can choose from programs in AI, Machine Learning, and Data Science. Would you like to know the course structure for these?

Adjusting for ambient noise... Please wait.
Please speak now...
Sorry, I couldn't understand the audio.
Sara: Hey! Glad you're here. What can I do for you?

Adjusting for ambient noise... Please wait.
Please speak now.